<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [1]:
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
file = files.upload()

Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn.csv


In [3]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [7]:
## Просмотр данных
def prosmotr(data):
  pd.set_option('display.max_columns', 100) #Размеры таблицы
  pd.set_option('display.max_rows', 100)
  pd.set_option('precision', 2) #Регулируем количество знаков после запятой:
  print('~~~~Содержание данных~~~~\n', data.head())
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('~~~Размеры данных~~~\n', data.shape)
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('~~~Названия колонок~~~\n', data.columns)
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('~~~Информация о данных~~~\n')
  print(data.info())
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('~~~Наличие пропусков в данных~~~\n', data.isna().sum())
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('~~~Количество типов в данных~~~')
  print(data.dtypes.value_counts())
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  kateg = list(data.select_dtypes(include=['object']).columns) # Делаем список категориальных данных
  print('~~~Категориальные данные~~~~')
  print(kateg)
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  chislov_float = list(data.select_dtypes(include=['float64'])) #Делаем список числовых данных float
  print('~~~Числове данные float~~~~')
  print(chislov_float)
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  chislov_int = list(data.select_dtypes(include=['int64'])) #Делаем список числовых данных int
  print('~~~Числове данные int~~~~')
  print(chislov_int)
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('~~~Основные статистические характеристики данных по каждому числовому признаку (типы int64)~~~\n', data.describe(include=['int64']))
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('~~~Основные статистические характеристики данных по каждому числовому признаку (типы float64)~~~\n', data.describe(include=['float64']))
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('~~~Cтатистика по нечисловым признакам object ~~~\n', data.describe(include=['object']))
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

общая информация о данных

In [9]:
prosmotr(df)

~~~~Содержание данных~~~~
    customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity OnlineBackup  \
0  No phone service             DSL             No          Yes   
1                No             DSL            Yes           No   
2                No             DSL            Yes          Yes   
3  No phone service             DSL            Yes           No   
4                No     Fiber optic             No           No   

  DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0          

##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [15]:
f = df.gender.loc[df['gender'] == 'Female'].count() 
m = df.gender.loc[df['gender'] == 'Male'].count()
print(f'female = {(f/(f + m))*100}, male = {(m/(f + m))*100}')

female = 49.5243504188556, male = 50.4756495811444


##### 2. Какое количество уникальных значений у поля InternetService?

In [17]:
len(set(df.InternetService.unique()))

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [18]:
df.TotalCharges.describe()

count     7043
unique    6531
top           
freq        11
Name: TotalCharges, dtype: object

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

In [19]:
df.TotalCharges = df.TotalCharges.apply(pd.to_numeric, errors='coerce')
df.TotalCharges.describe()

count    7032.00
mean     2283.30
std      2266.77
min        18.80
25%       401.45
50%      1397.47
75%      3794.74
max      8684.80
Name: TotalCharges, dtype: float64

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [21]:
df.PhoneService.replace(to_replace='Yes', value='1', inplace=True)
df.PhoneService.replace(to_replace='No', value='0', inplace=True)
df.PhoneService = pd.to_numeric(df.PhoneService)

In [22]:
df.PhoneService.head()

0    0
1    1
2    1
3    0
4    1
Name: PhoneService, dtype: int64

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [23]:
df['TotalCharges'] = df['TotalCharges'].astype('float32')
df['TotalCharges'].isnull().sum()

11

In [24]:
df['TotalCharges'] = df['TotalCharges'].fillna(0)
df.TotalCharges.describe()

count    7043.00
mean     2279.73
std      2266.79
min         0.00
25%       398.55
50%      1394.55
75%      3786.60
max      8684.80
Name: TotalCharges, dtype: float64

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [25]:
df.Churn.replace(to_replace='Yes', value='1', inplace=True)
df.Churn.replace(to_replace='No', value='0', inplace=True)
df.Churn = pd.to_numeric(df.Churn)
df.Churn.head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [27]:
df[['StreamingMovies', 'StreamingTV', 'TechSupport']] = df[['StreamingMovies', 'StreamingTV', 'TechSupport']].replace(to_replace={'Yes': 1, 'No': 0, 'No internet service': 0})

In [28]:
df[['StreamingMovies', 'StreamingTV', 'TechSupport']].head(15)

,StreamingMovies,StreamingTV,TechSupport
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,1
4,0,0,0
5,1,1,0
6,0,1,0
7,0,0,0
8,1,1,1
9,0,0,0


##### 8. Заполните пропуски в поле PhoneService значением 0

In [29]:
df.PhoneService.fillna(0)
df.PhoneService.describe()

count    7043.0
mean        0.9
std         0.3
min         0.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: PhoneService, dtype: float64

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [30]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']

df.drop(df.columns.difference(columns), 1, inplace=True)
df.head(3)

,gender,tenure,PhoneService,TechSupport,StreamingTV,StreamingMovies,TotalCharges,Churn
0,Female,1,0,0,0,0,29.85,0
1,Male,34,1,0,0,0,1889.50,0
2,Male,2,1,0,0,0,108.15,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [31]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'

X_train, X_test, y_train, y_test = train_test_split(df, df[target], test_size=0.33, shuffle=True, random_state=0)

In [32]:
X_test

,gender,tenure,PhoneService,TechSupport,StreamingTV,StreamingMovies,TotalCharges,Churn
2200,Female,19,1,0,1,0,1045.25,0
4627,Male,60,1,1,1,1,7049.50,0
3225,Female,13,1,0,1,1,923.85,0
2828,Male,1,1,0,0,0,20.45,0
3768,Female,55,1,0,0,0,4266.40,0
...,...,...,...,...,...,...,...,...
5407,Female,63,1,1,0,0,4014.20,0
3909,Male,11,1,0,0,0,223.15,0
6747,Female,52,0,0,0,0,1958.95,0
2774,Male,6,1,1,0,0,288.35,0


In [33]:
X_train

,gender,tenure,PhoneService,TechSupport,StreamingTV,StreamingMovies,TotalCharges,Churn
2337,Female,56,1,0,1,1,5510.65,1
2560,Male,46,1,0,0,1,4154.80,1
4725,Male,1,1,1,0,0,62.05,1
4278,Female,48,1,0,1,1,4692.95,0
3592,Female,1,1,0,0,0,45.65,1
...,...,...,...,...,...,...,...,...
4931,Male,15,1,1,1,1,1539.80,0
3264,Female,10,1,1,1,0,964.35,0
1653,Male,58,1,0,0,0,1185.95,0
2607,Male,1,1,0,0,0,69.75,1


In [34]:
y_test

2200    0
4627    0
3225    0
2828    0
3768    0
       ..
5407    0
3909    0
6747    0
2774    0
1177    0
Name: Churn, Length: 2325, dtype: int64

In [35]:
y_train

2337    1
2560    1
4725    1
4278    0
3592    1
       ..
4931    0
3264    0
1653    0
2607    1
2732    0
Name: Churn, Length: 4718, dtype: int64

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [36]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [38]:
gender.fit_transform(X_train)

,gender_Female,gender_Male
2337,1,0
2560,0,1
4725,0,1
4278,1,0
3592,1,0
...,...,...
4931,0,1
3264,1,0
1653,0,1
2607,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [39]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

Стандартизировать полезно признаки для модели, которая опирается на распределение (линейная модель).

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [40]:
TotalCharges = Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
])


Объединение всех "кубиков" очень легко сделать таким образом

In [41]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [42]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [43]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('tenure',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler(copy=True,
                                                                                 with_mean=True,
                                                                                 with_std=True))],
                                                          verbose=False)),
                                                ('TotalCharges',
                                                 Pipeline(memory=None,
                        

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [44]:
pipeline.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [45]:
len(X_test), len(pipeline.predict(X_test))

(2325, 2325)

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [46]:
from sklearn.metrics import roc_auc_score, log_loss

roc_auc_score(y_test, pipeline.predict(X_test)), log_loss(y_test, pipeline.predict(X_test))

(0.6562576229970575, 8.482506506535584)

### Сохраним наш пайплайн

In [47]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)

In [48]:
!ls

model_RF.dill  sample_data  WA_Fn-UseC_-Telco-Customer-Churn.csv
